In [ ]:
import pandas as pd
import BERT_model
from BERT_model import *

In [ ]:
import torch

model = torch.load('../data/model/largemodel_sample1000_epoch12.pt', map_location=torch.device('cpu'))
model.eval()

In [ ]:
import re

def prediction(model, text):
    #특수 문자 제거
    text_rmv = re.sub('[-=+,#/\?:^.@*\"※~ㆍ!』‘|\(\)\[\]`\'…》\”\“\’·]', ' ', text)
    # 공백 한개로 만들기
    new_str = ' '.join(text_rmv.split())
    # 임의의 dataframe으로 만들기
    text_dict = {'posts': [new_str], 'type' : ['INTP']}
    test_data = pd.DataFrame(text_dict)

    #훈련 모델에 맞는 Dataset으로 변환
    test = Dataset(test_data)
    test_dataloader = torch.utils.data.DataLoader(test, batch_size=2)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:

        model = model.cuda()

    total_acc_test = 0
    with torch.no_grad():

        for test_input, test_label in test_dataloader:
            output = BERT_model.GetModelOutput(model, test_input, test_label, device)
            predicted = output.argmax(dim=1)
            
    return predicted

In [ ]:
from googletrans import Translator

# 입력된 텍스트가 한글인지 영어인지 판단해서 한글이 하나라도 있으면 영어로 변역한다.
def isKorean(input_s):
    k_count = 0
    e_count = 0
    for c in input_s:
        if ord('가') <= ord(c) <= ord('힣'):
            k_count+=1
        elif ord('a') <= ord(c.lower()) <= ord('z'):
            e_count+=1

    return k_count > 0

# 한글에서 영어로 번역
def languageTrans(text):
    translator = Translator()

    translation = translator.translate(text, dest = 'en')

    return translation.text

In [ ]:
input_text = input('뭐든 입력해봐')

print(len(input_text))

if isKorean(input_text):
    input_text = languageTrans(input_text)

predicted = prediction(model, input_text)

print(BERT_model.resultLabels.get(predicted.item()))